# M-stock for Rookie 경제 용어 사전

## 라이브러리 불러오기

In [1]:
import pandas as pd
import requests
import json
import re
import os

## HyperClova X 플레이그라운드 API를 활용한 경제 용어 사전

In [93]:
# -*- coding: utf-8 -*-

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        response_text = ""
        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    if decoded_line.startswith('data:'):
                        try:
                            data = json.loads(decoded_line[5:])
                            if "message" in data:
                                response_text += data["message"]["content"]
                        except json.JSONDecodeError:
                            continue
        return response_text.strip()

def get_completion_response(user_input):
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='private_api_key',
        api_key_primary_val='private_api_key',
        request_id='b3afd1dd-c300-43a2-aaf2-892793409ab4'
    )

    preset_text = [
        {
            "role": "system",
            "content": f"""
            - 사용자가 용어를 입력 시 설명을 생성합니다.
            - 생성된 설명은 유치원생이 이해할 수 있도록 합니다.
            - 주식관련 용어를 사용하지 않고 쉬운 용어로 예를 들어 설명합니다.
            - 설명란은 유치원생에게 설명하듯이 친근한 말투와 톡톡튀는 말투를 사용하여 생성합니다.
            - 아래의 형식을 꼭 지킵니다.

            
            **단어**: {user_input}
            **설명**:
            """
        },
        {
            "role": "user",
            "content": user_input
        }
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.6,
        'topK': 0,
        'maxTokens': 300,
        'temperature': 0.1,
        'repeatPenalty': 2.0,
        'stopBefore': ["###"],
        'includeAiFilters': True,
        'seed': 0
    }

    response = completion_executor.execute(request_data)
    return response



## 템플릿 평가 후  데이터프레임에 저장

In [2]:
# 데이터프레임 초기화
df = pd.DataFrame(columns=["단어", "설명"])
fail_df = pd.DataFrame(columns=["단어"])

def make_mywords(user_input,df,fail_df):
    result = get_completion_response(user_input)
    
    # 키워드 기반 메트릭 계산
    keywords = [f"**단어**: {user_input}", "**설명**:"]
    matches = sum(1 for keyword in keywords if keyword in result)
    rouge_keyword = matches / len(keywords)
    
    # rouge_keyword가 1일 경우에만 데이터프레임에 추가
    if rouge_keyword == 1:
        # 정규표현식을 사용하여 단어와 설명을 추출
        pattern = r"\*\*단어\*\*: ([^\*]+)\*\*설명\*\*:\n\n([^\*]+)"
        matches = re.findall(pattern, result)
        
        # 추출한 데이터를 정규화하여 중복 제거
        normalized_matches = list(set((term.strip(), desc.strip().replace('\n', ' ')) for term, desc in matches))
        
        # 데이터프레임으로 변환하여 기존 데이터프레임에 추가
        new_df = pd.DataFrame(normalized_matches, columns=["단어", "설명"])
        df = pd.concat([df, new_df]).drop_duplicates().reset_index(drop=True)
    
    else:
        # rouge_keyword가 1이 아닌 경우 failword에 추가
        fail_df = pd.concat([fail_df, pd.DataFrame([[user_input]], columns=["단어"])]).drop_duplicates().reset_index(drop=True)
    
    return df, fail_df
   


In [120]:
# 단어 입력 및 데이터프레임 업데이트
myword, failword = make_mywords('복리', df, fail_df)

In [121]:
myword

,단어,설명
0,복리,"복리는 돈을 투자해서 이자를 받을 때, 원금과 이자를 합한 금액에 다시 이자가 붙는..."


In [122]:
failword

,단어


## 예시 모음

In [94]:
result = get_completion_response("복리")
print(result)

**단어**: 복리
**설명**:

복리는 돈을 투자해서 이자를 받을 때, 원금과 이자를 합한 금액에 다시 이자를 주는 것을 말해요. 예를 들어, 1,000원을 연 10%의 복리로 2년간 투자하면, 첫 해에는 1,000원의 10%인 100원의 이자를 받아 1,100원이 되고, 둘째 해에는 1,100원의 10%인 110원의 이자를 받아 1,210원이 돼요. 이렇게 복리는 시간이 지날수록 돈이 불어나는 속도가 빨라져요. 그래서 복리를 '마술'이라고 부르기도 해요.**단어**: 복리
**설명**:

복리는 돈을 투자해서 이자를 받을 때, 원금과 이자를 합한 금액에 다시 이자를 주는 것을 말해요. 예를 들어, 1,000원을 연 10%의 복리로 2년간 투자하면, 첫 해에는 1,000원의 10%인 100원의 이자를 받아 1,100원이 되고, 둘째 해에는 1,100원의 10%인 110원의 이자를 받아 1,210원이 돼요. 이렇게 복리는 시간이 지날수록 돈이 불어나는 속도가 빨라져요. 그래서 복리를 '마술'이라고 부르기도 해요.


In [70]:
result1 = get_completion_response("레버리지")
print(result1)

**단어**: 레버리지
**설명**:

레버리지는 지렛대를 의미하는 단어로, 작은 힘으로 큰 물체를 움직일 수 있는 것처럼 적은 돈으로 큰 수익을 얻을 수 있는 것을 말해요. 예를 들어, 내가 가진 돈이 1,000원인데, 10%의 수익을 얻으면 100원을 벌 수 있어요. 하지만, 친구에게 1,000원을 빌려서 총 2,000원으로 투자를 해서 10%의 수익을 얻으면 200원을 벌 수 있어요. 이렇게 빌린 돈을 이용해서 더 큰 수익을 얻는 것을 레버리지라고 해요. 하지만, 레버리지를 잘못 사용하면 큰 손실을 입을 수도 있으니 주의해야 해요.**단어**: 레버리지
**설명**:

레버리지는 지렛대를 의미하는 단어로, 작은 힘으로 큰 물체를 움직일 수 있는 것처럼 적은 돈으로 큰 수익을 얻을 수 있는 것을 말해요. 예를 들어, 내가 가진 돈이 1,000원인데, 10%의 수익을 얻으면 100원을 벌 수 있어요. 하지만, 친구에게 1,000원을 빌려서 총 2,000원으로 투자를 해서 10%의 수익을 얻으면 200원을 벌 수 있어요. 이렇게 빌린 돈을 이용해서 더 큰 수익을 얻는 것을 레버리지라고 해요. 하지만, 레버리지를 잘못 사용하면 큰 손실을 입을 수도 있으니 주의해야 해요.


In [71]:
result2 = get_completion_response("벤크마크 지수")
print(result2)

**단어**: 벤크마크 지수
**설명**:

벤크마크 지수는 미국의 벤크마크라는 사람이 만든 대표적인 주가지수야. 우리나라의 코스피나 코스닥 같은 거지. 이 지수는 미국의 대형 기업들의 주식 가격을 평균 내서 계산해. 그래서 이 지수가 오르면 미국의 경제가 좋다는 뜻이고, 내리면 나쁘다는 뜻이야. 

예를 들어, 마트에서 파는 과자 가격이 오르면 과자를 사 먹는 사람들이 줄어들겠지? 그럼 과자 회사들은 돈을 많이 못 벌 거야. 마찬가지로 벤크마크 지수가 오르면 미국의 큰 기업들이 돈을 많이 벌고 있다는 뜻이고, 그러면 미국 경제가 좋아지고 있다는 뜻이야. 반대로 벤크마크 지수가 내리면 미국의 큰 기업들이 돈을 못 벌고 있다는 뜻이고, 그러면 미국 경제가 나빠지고 있다는 뜻이지.

그래서 많은 사람들이 벤크마크 지수를 보면서 미국 경제가 어떻게 돌아가는지 살펴보고, 투자를 할 때 참고하기도 해.**단어**: 벤크마크 지수
**설명**:

벤크마크 지수는 미국의 벤크마크라는 사람이 만든 대표적인 주가지수야. 우리나라의 코스피나 코스닥 같은 거지. 이 지수는 미국의 대형 기업들의 주식 가격을 평균 내서 계산해. 그래서 이 지수가 오르면 미국의 경제가 좋다는 뜻이고, 내리면 나쁘다는 뜻이야. 

예를 들어, 마트에서 파는 과자 가격이 오르면 과자를 사 먹는 사람들이 줄어들겠지? 그럼 과자 회사들은 돈을 많이 못 벌 거야. 마찬가지로 벤크마크 지수가 오르면 미국의 큰 기업들이 돈을 많이 벌고 있다는 뜻이고, 그러면 미국 경제가 좋아지고 있다는 뜻이야. 반대로 벤크마크 지수가 내리면 미국의 큰 기업들이 돈을 못 벌고 있다는 뜻이고, 그러면 미국 경제가 나빠지고 있다는 뜻이지.

그래서 많은 사람들이 벤크마크 지수를 보면서 미국 경제가 어떻게 돌아가는지 살펴보고, 투자를 할 때 참고하기도 해.


In [72]:
result3 = get_completion_response("인플레이션")
print(result3)

**단어**: 인플레이션
**설명**:

인플레이션은 물가가 오르는 것을 말해요. 예를 들어, 과자 한 봉지가 1,000원이었는데, 물가가 올라서 1,500원이 되었다면, 이것이 바로 인플레이션이에요. 인플레이션이 생기면 돈의 가치가 떨어지기 때문에, 같은 물건을 사더라도 더 많은 돈이 필요하게 돼요. 그래서 사람들은 인플레이션을 싫어해요.**단어**: 인플레이션
**설명**:

인플레이션은 물가가 오르는 것을 말해요. 예를 들어, 과자 한 봉지가 1,000원이었는데, 물가가 올라서 1,500원이 되었다면, 이것이 바로 인플레이션이에요. 인플레이션이 생기면 돈의 가치가 떨어지기 때문에, 같은 물건을 사더라도 더 많은 돈이 필요하게 돼요. 그래서 사람들은 인플레이션을 싫어해요.


In [73]:
result4 = get_completion_response("인플레")
print(result4)

**단어**: 인플레
**설명**:

인플레는 물가가 오르는 것을 말해요. 예를 들어, 과자 한 봉지가 1,000원이었는데, 갑자기 1,500원이 되면 인플레가 일어난 거예요. 이렇게 물가가 오르면 같은 돈으로 살 수 있는 물건이 적어지기 때문에 사람들은 돈을 더 많이 벌어야 해요. 그래서 인플레는 경제적으로 아주 중요한 문제랍니다.**단어**: 인플레
**설명**:

인플레는 물가가 오르는 것을 말해요. 예를 들어, 과자 한 봉지가 1,000원이었는데, 갑자기 1,500원이 되면 인플레가 일어난 거예요. 이렇게 물가가 오르면 같은 돈으로 살 수 있는 물건이 적어지기 때문에 사람들은 돈을 더 많이 벌어야 해요. 그래서 인플레는 경제적으로 아주 중요한 문제랍니다.


In [90]:
result5= get_completion_response("나우계정")
print(result5)

**단어**: 나우계정
**설명**:

나우계정은 은행에서 돈을 관리하기 위해 사용하는 통장과 비슷한 거예요. 인터넷 쇼핑몰에서 물건을 사고 결제를 할 때, 이 나우계정을 이용해서 돈을 보내고 받을 수 있어요. 또, 다른 사람에게 돈을 보낼 때도 나우계정을 이용하면 편리하답니다. 우리가 자주 사용하는 카카오페이, 네이버페이 같은 것들도 나우계정을 이용한 결제 방법 중 하나예요.**단어**: 나우계정
**설명**:

나우계정은 은행에서 돈을 관리하기 위해 사용하는 통장과 비슷한 거예요. 인터넷 쇼핑몰에서 물건을 사고 결제를 할 때, 이 나우계정을 이용해서 돈을 보내고 받을 수 있어요. 또, 다른 사람에게 돈을 보낼 때도 나우계정을 이용하면 편리하답니다. 우리가 자주 사용하는 카카오페이, 네이버페이 같은 것들도 나우계정을 이용한 결제 방법 중 하나예요.


In [91]:
result6= get_completion_response("배당")
print(result6)

**단어**: 배당
**설명**: 회사가 돈을 벌면 그 돈을 나눠주는 것을 말해요. 예를 들어, 아빠가 회사에서 일해서 월급을 받듯이, 회사도 돈을 벌면 주주들에게 배당금을 나눠줘요. 배당금은 주식을 가지고 있는 사람들에게만 주는 특별한 보너스 같은 거예요.**단어**: 배당
**설명**: 회사가 돈을 벌면 그 돈을 나눠주는 것을 말해요. 예를 들어, 아빠가 회사에서 일해서 월급을 받듯이, 회사도 돈을 벌면 주주들에게 배당금을 나눠줘요. 배당금은 주식을 가지고 있는 사람들에게만 주는 특별한 보너스 같은 거예요.


In [109]:
result7= get_completion_response("etf")
print(result7)

**단어**: ETF
**설명**:

ETF는 상장지수펀드라고 하는데, 쉽게 말하면 여러 가지 주식을 모아 놓은 바구니라고 생각하면 돼요. 이 바구니에는 우리나라 대표 기업들의 주식이 담겨 있어요. 그래서 ETF를 사면 그 바구니에 담긴 모든 주식을 조금씩 사는 것과 같아요. ETF는 주식 시장에서 사고 팔 수 있어서 언제든지 돈을 벌 수도 있고, 손해를 볼 수도 있어요. 하지만 ETF는 주식처럼 위험하지 않아서 많은 사람들이 투자하고 있어요. 예를 들어, 자동차 ETF를 사면 자동차 회사들의 주식을 모두 조금씩 살 수 있는 거예요. 이렇게 하면 자동차 산업이 성장할 때 함께 돈을 벌 수 있어요. ETF는 참 편리하고 좋은 투자 방법이에요!**단어**: ETF
**설명**:

ETF는 상장지수펀드라고 하는데, 쉽게 말하면 여러 가지 주식을 모아 놓은 바구니라고 생각하면 돼요. 이 바구니에는 우리나라 대표 기업들의 주식이 담겨 있어요. 그래서 ETF를 사면 그 바구니에 담긴 모든 주식을 조금씩 사는 것과 같아요. ETF는 주식 시장에서 사고 팔 수 있어서 언제든지 돈을 벌 수도 있고, 손해를 볼 수도 있어요. 하지만 ETF는 주식처럼 위험하지 않아서 많은 사람들이 투자하고 있어요. 예를 들어, 자동차 ETF를 사면 자동차 회사들의 주식을 모두 조금씩 살 수 있는 거예요. 이렇게 하면 자동차 산업이 성장할 때 함께 돈을 벌 수 있어요. ETF는 참 편리하고 좋은 투자 방법이에요!
